In [ ]:
from tqdm.auto import tqdm

from utils.emit import ortho_gbox
from utils.txt import from_njson

In [ ]:
docs = list(tqdm(from_njson("Data/emit-zarr-meta.njson.gz")))

In [ ]:
doc = docs[300]
url = doc["url"].rsplit(".", 1)[0]
zz = doc["zarr"]
url, list(zz[".zattrs"]), list(zz)

In [ ]:
props_keep = [
    "sensor",
    "instrument",
    "platform",
    "date_created",
    "flight_line",
    "time_coverage_start",
    "time_coverage_end",
    "crosstrack_orientation",
    "spatialResolution",
    "spatial_ref",
    "geotransform",
    "day_night_flag",
]

bbox_keys = [
    "westernmost_longitude",
    "southernmost_latitude",
    "easternmost_longitude",
    "northernmost_latitude",
]

renames = {
    "time_coverage_start": "start_datetime",
    "time_coverage_end": "end_datetime",
    "spatialResolution": "gsd",
}


def extract_md(doc):
    url = doc["url"].rsplit(".", 1)[0]
    zz = doc["zarr"]
    attrs = zz[".zattrs"]
    props = {renames.get(k, k): attrs[k] for k in props_keep}
    bbox = [attrs[k] for k in bbox_keys]

    h, w, b = zz["reflectance/.zarray"]["shape"]
    props.update(
        width=w,
        height=h,
        nbands=b,
        datetime=props["start_datetime"],
    )
    props["proj:epsg"] = 4326

    return {"href": url, "bbox": bbox, "properties": props}


extract_md(doc)

In [ ]:
ortho_gbox(doc["zarr"])

In [ ]:
zz

In [ ]:
%%bash

source .env

curl -H "Authorization: Bearer $EARTHDATA_TOKEN" \
  "https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials" -o s3_creds.json

#hash jq && jq . < s3_creds.json

In [ ]:
!jq . s3_creds.json

In [ ]:
%%bash

source .env

prod="https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001"
granule="EMIT_L2A_RFL_001_20230531T133036_2315109_002/EMIT_L2A_RFL_001_20230531T133036_2315109_002.nc"
outfile="${granule##*/}"
url="${prod}/${granule}"
printf "url: %s\nout: %s\ntk: %s...\n\n" $url $outfile ${EARTHDATA_TOKEN:0:8}

curl -v -H "Authorization: Bearer $EARTHDATA_TOKEN" $url 

In [ ]:
%%bash

source .env

prod="https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001"
granule="EMIT_L2A_RFL_001_20230531T133036_2315109_002/EMIT_L2A_RFL_001_20230531T133036_2315109_002.nc"
outfile="${granule##*/}"
url="${prod}/${granule}"
printf "url: %s\nout: %s\ntk: %s...\n\n" $url $outfile ${EARTHDATA_TOKEN:0:8}

echo curl -L -H "Authorization: Bearer $EARTHDATA_TOKEN" $url -o $outfile